In [1]:
import cv2
import numpy as np
import dlib
import face_recognition
import matplotlib.pyplot as plt
import pandas as pd
import os
import sys
import glob
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing import image

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")


face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
lip_cascade = cv2.CascadeClassifier('haarcascade_smile.xml')



In [2]:
faces = []

In [65]:
## to view/save picture

video_capture = cv2.VideoCapture(1, cv2.IMREAD_GRAYSCALE)

while True:
    # Capture frame-by-frame
    ret, img = video_capture.read()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 

#     img = cv2.imread('images/test2/surprise/0_32.jpg', cv2.IMREAD_GRAYSCALE)
    dets = detector(img)
    for k, d in enumerate(dets):
        shape = predictor(img, d)
        xmouthpoints = [shape.part(x).x for x in range(0,68)]
        ymouthpoints = [shape.part(x).y for x in range(0,68)]
        maxx = max(xmouthpoints)
        minx = min(xmouthpoints)
        maxy = max(ymouthpoints)
        miny = min(ymouthpoints) 

        crop_image = img[miny:maxy,minx:maxx]
        crop_image = cv2.resize(crop_image, (224,224))
    
        cv2.imshow('face', crop_image)
        cv2.imshow('original', img)
    if cv2.waitKey(1) &  0xFF == ord('a'): 
        print('yeeted')
        faces.append(crop_image)

    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
    
    
cv2.destroyAllWindows()

In [68]:
for f in faces:
    img = cv2.cvtColor(f, cv2.COLOR_BGR2RGB)
    im = Image.fromarray(img)
#     im.save(f"H_{i}.jpg")
    i += 1

In [265]:
model = tf.keras.models.load_model('saveModel') 

In [262]:
def lableToName(label: str):
    if label==0:
        return 'joy'
    elif label==1:
        return 'neutral'
    elif label==2:
        return 'surprise'

In [3]:
def getFace(img):
    faces = detector(img)
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

        landmarks = predictor(img, face)
    pts = []
    for n in range(0, 16):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        pts.append((x, y))
    for n in range(26, 21, -1):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        pts.append((x, y))
    for n in range(21, 16, -1):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        pts.append((x, y))

    x = landmarks.part(0).x
    y = landmarks.part(0).y
    pts.append((x, y))

    pts = np.array(pts)

    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    croped = img[y:y+h, x:x+w].copy()
#     pts = pts - pts.min(axis=0)

#     mask = np.zeros(croped.shape[:2], np.uint8)
#     cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
#     dst = cv2.bitwise_and(croped, croped, mask=mask)

#     ## (4) add the white background
#     bg = np.ones_like(croped, np.uint8)*255
#     cv2.bitwise_not(bg,bg, mask=mask)
#     dst2 = bg+ dst

    return dst

In [252]:
def getLips(img):
    faces = detector(img)
    for face in faces:
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        #cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)

        landmarks = predictor(img, face)
    pts = []
    for n in range(48, 60):
        x = landmarks.part(n).x
        y = landmarks.part(n).y
        pts.append((x, y))
    
    x = landmarks.part(48).x
    y = landmarks.part(48).y
    pts.append((x, y))

    pts = np.array(pts)

    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    croped = img[y:y+h, x:x+w].copy()
    
#     pts = pts - pts.min(axis=0)

#     mask = np.zeros(croped.shape[:2], np.uint8)
#     cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
#     dst = cv2.bitwise_and(croped, croped, mask=mask)

#     ## (4) add the white background
#     bg = np.ones_like(croped, np.uint8)*255
#     cv2.bitwise_not(bg,bg, mask=mask)
#     dst2 = bg+ dst
    return croped

In [253]:
def lableToName(label: str):
    if label==0:
        return 'joy'
    elif label==1:
        return 'neutral'
    elif label==2:
        return 'surprise'

In [254]:
lips_points = []
labels = []

In [255]:
"""
0 -> neutral
1 -> joy
2 -> surprise

"""

'\n0 -> neutral\n1 -> joy\n2 -> surprise\n\n'

In [256]:
model = tf.keras.models.load_model('saveModel') 

In [283]:
img_path = 'images/final/surprise/'
save_path = 'images/lips/surprise/'
for i in os.listdir(img_path):
    img = cv2.imread(img_path+i)
    try:
        lips_img = getLips(img)    
        lips_img = cv2.resize(lips_img, (64,64))
        lips_img = cv2.cvtColor(lips_img, cv2.COLOR_BGR2GRAY) 
        im = Image.fromarray(lips_img)
        im.save(save_path+i)
#         lips_points.append(lips_img)
#         labels.append(2)
    except:
        print('no face detected in ', i)
    
#     face_img = getFace(img)
#     print(i)
#     plt.imshow(lips_img)
#     plt.show()

no face detected in  12_41.jpg
no face detected in  1_34.jpg
no face detected in  8_67.jpg
no face detected in  H_10.jpg
no face detected in  H_11.jpg
no face detected in  H_21.jpg
no face detected in  H_22.jpg
no face detected in  H_23.jpg
no face detected in  H_24.jpg
no face detected in  H_9.jpg


In [86]:
video_capture = cv2.VideoCapture(1, cv2.IMREAD_GRAYSCALE)

while True:
    # Capture frame-by-frame
    ret, img_frame = video_capture.read()
    img_frame = cv2.cvtColor(img_frame, cv2.COLOR_BGR2GRAY) 
    
    
    try:
        dets = detector(img_frame)
    except:
        pass
    
    for k, d in enumerate(dets):
        shape = predictor(img_frame, d)
        xmouthpoints = [shape.part(x).x for x in range(0,68)]
        ymouthpoints = [shape.part(x).y for x in range(0,68)]
        maxx = max(xmouthpoints)
        minx = min(xmouthpoints)
        maxy = max(ymouthpoints)
        miny = min(ymouthpoints) 

        crop_image_ = img_frame[miny:maxy,minx:maxx]
        crop_image = cv2.resize(crop_image_, (224,224))
        
        cv2.imshow('face', crop_image)
        cv2.imshow('non-resized', crop_image_)

        cv2.imshow('camera feed', img_frame)
    
        img_array = image.img_to_array(crop_image)
        img_batch = np.expand_dims(img_array, axis=0)
        emo = np.argmax(model.predict(img_batch))
        print(lableToName(emo))
    
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyAllWindows()

surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
joy
joy
joy
surprise
surprise
joy
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
joy
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
joy
joy
joy
joy
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
surprise
s

In [267]:
video_capture = cv2.VideoCapture(1)

while True:
    # Capture frame-by-frame
    ret, img = video_capture.read()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
    
    lips_img = getLips(img)    
    
    lips_img = cv2.resize(lips_img, (64,64))

    cv2.imshow('face', lips_img)

    cv2.imshow('camera feed', img)

    img_array = image.img_to_array(lips_img)
    img_batch = np.expand_dims(img_array, axis=0)
    emo = np.argmax(model.predict(img_batch))
    
    print(lableToName(emo))
#     print(emo)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

cv2.destroyAllWindows()

neutral
